In [1]:
import random
import os
import glob
import shutil

In [2]:
random_country_dir = random.choice(os.listdir("../country_data"))
random_country_dir

'COG_Congo'

In [3]:
figures = glob.glob(f"../country_data/{random_country_dir}/figures/*.png", recursive = True)
figures

['../country_data/COG_Congo/figures/COG_CAIT_gases_1.png',
 '../country_data/COG_Congo/figures/COG_CO2_totals.png',
 '../country_data/COG_Congo/figures/COG_GCP_1.png',
 '../country_data/COG_Congo/figures/COG_EIA_1.png',
 '../country_data/COG_Congo/figures/COG_UNFCCC_NAI_1.png',
 '../country_data/COG_Congo/figures/COG_CDIAC_1.png',
 '../country_data/COG_Congo/figures/COG_IEA_1.png',
 '../country_data/COG_Congo/figures/COG_Minx_top20_subsectors.png']

In [4]:
random_fig = random.choice(figures)
random_fig

'../country_data/COG_Congo/figures/COG_CO2_totals.png'

In [5]:
shutil.copyfile(random_fig, "../supplemental/random_daily_fig.png",)

'../supplemental/random_daily_fig.png'

In [6]:
print("random_fig path imported")

random_fig path imported
